In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# import sleap, get confidence maps on some nice data, etc...

In [3]:
import os
import io
import copy
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import toml
import h5py
from tqdm.auto import tqdm
from markovids import vid

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
save_dir = "/mnt/data/jmarkow/panels/2024-06 (QD paper)"

In [5]:
preview_dir = "/home/jmarkow/data_dir/active_projects/quantum_dots/timecourse_02/session_20240610164015-424335 (system76-pc)/_proc/"
# preview_dir = "/home/jmarkow/data_dir/active_projects/quantum_dots/timecourse_02_joints/session_20240807185653-046788 (system76-pc)/_proc/"

In [6]:
timestamps = pd.read_csv(os.path.join(preview_dir, "timestamps-fluorescence.txt"), delimiter="\s+")

In [7]:
reflect_files = sorted(glob.glob(os.path.join(preview_dir, "*reflectance.avi")))
cameras = [os.path.basename(_).replace("-reflectance.avi","") for _ in reflect_files]
fluo_files = [_.replace("-reflectance.avi","-fluorescence.avi") for _ in reflect_files] 

In [8]:
ts_paths = {os.path.join(preview_dir, "..", f"{_cam}.txt"): _cam for _cam in cameras}

In [9]:
ts, use_merged_ts = vid.io.read_timestamps_multicam(ts_paths, merge_tolerance=0.001)
use_merged_ts = use_merged_ts.dropna()
ts_fluo = use_merged_ts.loc[
    np.mod(use_merged_ts.index, 2) == 0
]  # fluorescence is 0,2,4,etc..
ts_reflect = use_merged_ts.loc[
    np.mod(use_merged_ts.index, 2) == 1
]  # reflectance is 1,3,5,...

ts_idx = ts_reflect.index.get_indexer(ts_fluo.index, method="nearest", tolerance=2)
ts_include = np.flatnonzero(ts_idx >= 0)  # -1 if we don't get a match
ts_fluo = ts_fluo.iloc[ts_include]
ts_reflect = ts_reflect.iloc[
    ts_reflect.index.get_indexer(ts_fluo.index, method="nearest", tolerance=2)
]

In [10]:
effective_fs = (1 / ts_fluo["system_timestamp"].diff()).median().round(0)

In [11]:
# read in data from all cameras, reflectance and fluorescence...

In [12]:
preview_frames = range(1200) if "joint" not in preview_dir else range(4500,5700)

In [13]:
from collections import defaultdict

In [14]:
reflect_frames = {}
fluo_frames = {}
fluo_bground_sub_frames = {}

for _reflect, _fluo in tqdm(zip(reflect_files, fluo_files)):
    # 
    _camera = os.path.basename(_reflect).replace("-reflectance.avi","")
    
    reader = vid.io.AutoReader(_reflect)
    reflect_frames[_camera] = reader.get_frames(preview_frames)
    reader.close()

    # load and subtract background...
    dirname, basename = os.path.split(_fluo)
    new_basename = os.path.splitext(basename)[0] + ".hdf5"
    bground_fname = os.path.join(dirname, "_bground", new_basename)

    with h5py.File(bground_fname, "r") as f:
        bgrounds = f["bground"][()].astype("int16")
        bgrounds_idxs = f["frame_idxs"][()]
    
    reader = vid.io.AutoReader(_fluo)
    fluo_frames[_camera] = reader.get_frames(preview_frames)
    reader.close() 

    fluo_frames[_camera] = fluo_frames[_camera].astype("int16")
    for i, _frame_idx in enumerate(preview_frames):
        use_bground_idx = np.abs(_frame_idx - bgrounds_idxs).argmin()
        use_bground = bgrounds[use_bground_idx]
        fluo_frames[_camera][i] -= use_bground

    fluo_frames[_camera] = np.clip(fluo_frames[_camera], 0, 255).astype("uint8")

0it [00:00, ?it/s]

In [15]:
lut_reflect = {"vmin":0 ,"vmax": 255, "cmap": plt.get_cmap("grey")}
lut_fluo = {"vmin": 0, "vmax": 255, "cmap": plt.get_cmap("inferno")}

In [16]:
batch_size = 10
ncols = 3

In [17]:
nrows = np.ceil(len(cameras) / ncols).astype("int")

In [18]:
batches = range(0, len(preview_frames), batch_size)

In [19]:
# we'll concat across width...

In [20]:
_, height, width = list(fluo_frames.values())[0].shape

In [21]:
new_height = height * nrows
new_width = width * ncols * 2

In [22]:
fname = "sup_vid_00_beads_rigv2.mp4" if "joint" not in preview_dir else "sup_vid_00_beads_kneejoint_rigv2.mp4"

In [23]:
# INSERT FLUO AND REFLECT ONLY VIDS HERE...

In [24]:
fname_fluo = f"{os.path.splitext(fname)[0]}_fluo.mp4"
fname_reflect = f"{os.path.splitext(fname)[0]}_reflect.mp4"

In [25]:
writer = vid.io.MP4WriterPreview(
    os.path.join(save_dir, fname),
    frame_size=(new_width, new_height),
    fps=effective_fs,
)

writer_fluo = vid.io.MP4WriterPreview(
    os.path.join(save_dir, fname_fluo),
    frame_size=(new_width // 2, new_height),
    fps=effective_fs,
)
writer_reflect = vid.io.MP4WriterPreview(
    os.path.join(save_dir, fname_reflect),
    frame_size=(new_width // 2, new_height),
    fps=effective_fs,
)

In [26]:
for _batch in tqdm(batches):
    frame_batch = range(_batch, _batch + batch_size)
    reflect_frames_pseudo = [
        vid.io.pseudocolor_frames(_frames[frame_batch], **lut_reflect)
        for _frames in reflect_frames.values()
    ]
    fluo_frames_pseudo = [
        vid.io.pseudocolor_frames(_frames[frame_batch], **lut_fluo)
        for _frames in fluo_frames.values()
    ]
    # can batch this out...
    
    reflect_montage = vid.util.video_montage(reflect_frames_pseudo, ncols=3)
    fluo_montage = vid.util.video_montage(fluo_frames_pseudo, ncols=3)
    concat_frames = np.concatenate([reflect_montage, fluo_montage], axis=2)
    writer_fluo.write_frames(fluo_montage, inscribe_frame_number=False)
    writer_reflect.write_frames(reflect_montage, inscribe_frame_number=False)
    writer.write_frames(concat_frames, inscribe_frame_number=False)

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [27]:
writer_fluo.close()
writer_reflect.close()
writer.close()